# Terminal Processing

© Crown-owned copyright 2025, Defence Science and Technology Laboratory UK

This notebook serves as a guide on the functionality and use of the new Terminal simulation component.

The Terminal service comes pre-installed on most Nodes (The exception being Switches, as these are currently dumb). 

In [ ]:
!primaite setup

In [ ]:
from primaite.simulator.system.services.terminal.terminal import Terminal
from primaite.simulator.network.container import Network
from primaite.simulator.network.hardware.nodes.host.computer import Computer
from primaite.simulator.system.applications.red_applications.ransomware_script import RansomwareScript
from primaite.simulator.system.services.terminal.terminal import RemoteTerminalConnection

def basic_network() -> Network:
    """Utility function for creating a default network to demonstrate Terminal functionality"""
    network = Network()
    node_a = Computer.from_config(
            config = {
            "type": "computer",
            "hostname": "node_a",
            "ip_address": "192.168.0.10",
            "subnet_mask": "255.255.255.0",
            # "startup_duration": 0,
        }
    )
    print(f"{node_a=}")
    node_a.power_on()
    node_b = Computer.from_config(
        config = {
            "type": "computer",
            "hostname": "node_b",
            "ip_address": "192.168.0.11",
            "subnet_mask": "255.255.255.0",
            # "startup_duration": 0,
        }
    )
    node_b.power_on()
    network.connect(node_a.network_interface[1], node_b.network_interface[1])
    return network

The terminal can be accessed from a `Node` via the `software_manager` as demonstrated below. 

In the example, we have a basic network consisting of two computers, connected to form a basic network.

In [ ]:
network: Network = basic_network()
computer_a: Computer = network.get_node_by_hostname("node_a")
terminal_a: Terminal = computer_a.software_manager.software.get("terminal")
computer_b: Computer = network.get_node_by_hostname("node_b")
terminal_b: Terminal = computer_b.software_manager.software.get("terminal")

To be able to send commands from `node_a` to `node_b`, you will need to `login` to `node_b` first, using valid user credentials. In the example below, we are remotely logging in to the 'admin' account on `node_b`, from `node_a`. 
If you are not logged in, any commands sent will be rejected by the remote.

Remote Logins return a RemoteTerminalConnection object, which can be used for sending commands to the remote node. 

In [ ]:
# Login to the remote (node_b) from local (node_a)
term_a_term_b_remote_connection: RemoteTerminalConnection = terminal_a.login(username="admin", password="admin", ip_address="192.168.0.11")

You can view all active connections to a terminal through use of the `show()` method.

In [ ]:
terminal_b.show()

The new connection object allows us to forward commands to be executed on the target node. The example below demonstrates how you can remotely install an application on the target node.

In [ ]:
term_a_term_b_remote_connection.execute(["software_manager", "application", "install", "ransomware-script"])

In [ ]:
computer_b.software_manager.show()

The code block below demonstrates how the Terminal class allows the user of `terminal_a`, on `computer_a`, to send a command to `computer_b` to create a downloads folder. 


In [ ]:
# Display the current state of the file system on computer_b
computer_b.file_system.show()

# Send command
term_a_term_b_remote_connection.execute(["file_system", "create", "folder", "downloads"])

The resultant call to `computer_b.file_system.show()` shows that the new folder has been created.

In [ ]:
computer_b.file_system.show()

When finished, the connection can be closed by calling the `disconnect` function of the Remote Client object

In [ ]:
# Display active connection
terminal_a.show()
terminal_b.show()

term_a_term_b_remote_connection.disconnect()

terminal_a.show()
terminal_b.show()

Disconnected Terminal sessions will no longer show in the node's Terminal connection list, but will be under the historic sessions in the `user_session_manager`.

In [ ]:
computer_b.user_session_manager.show(include_historic=True, include_session_id=True)